In [14]:
!nvidia-smi

Sat Mar  4 02:03:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0    46W / 163W |   3378MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
# sync python module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Gen pretrain data dict json

In [44]:
workspace_dir = '/nfs/Workspace/CardiacSeg'
import sys

# set package path
sys.path.append(workspace_dir)

from datasets.chgh_dataset import get_data_dicts
from data_utils.io import save_json
from expers.preprocess import get_rel_data_dicts

exp_name = 'exp_2_2'
root_data_dir = os.path.join(workspace_dir, 'dataset', 'chgh')
data_dir = os.path.join(root_data_dir, 'dataset_2')
data_dicts_json = os.path.join(root_data_dir, 'data_dicts', f'{exp_name}_pretrain.json')

data_dicts = get_rel_data_dicts(get_data_dicts(data_dir))
data_dicts.pop(11)
data_dicts.pop(-3)
data_dicts[-5:-2]

# data_dicts

tr_data_dicts = [*data_dicts[:-5], *data_dicts[-2:]]
val_data_dicts = data_dicts[-5:-2]
tt_data_dicts = data_dicts[-5:-2]

out_data_dict = {
    'train':tr_data_dicts,
    'val':val_data_dicts,
    'test':tt_data_dicts
}

save_json(out_data_dict, data_dicts_json)
out_data_dict

save json to /nfs/Workspace/CardiacSeg/dataset/chgh/data_dicts/exp_2_2_pretrain.json


{'train': [{'image': 'pid_05/pid_05.nii.gz',
   'label': 'pid_05/pid_05_gt.nii.gz'},
  {'image': 'pid_06/pid_06.nii.gz', 'label': 'pid_06/pid_06_gt.nii.gz'},
  {'image': 'pid_07/pid_07.nii.gz', 'label': 'pid_07/pid_07_gt.nii.gz'},
  {'image': 'pid_08/pid_08.nii.gz', 'label': 'pid_08/pid_08_gt.nii.gz'},
  {'image': 'pid_21/pid_21.nii.gz', 'label': 'pid_21/pid_21_gt.nii.gz'},
  {'image': 'pid_23/pid_23.nii.gz', 'label': 'pid_23/pid_23_gt.nii.gz'},
  {'image': 'pid_24/pid_24.nii.gz', 'label': 'pid_24/pid_24_gt.nii.gz'},
  {'image': 'pid_27/pid_27.nii.gz', 'label': 'pid_27/pid_27_gt.nii.gz'},
  {'image': 'pid_30/pid_30.nii.gz', 'label': 'pid_30/pid_30_gt.nii.gz'},
  {'image': 'pid_34/pid_34.nii.gz', 'label': 'pid_34/pid_34_gt.nii.gz'},
  {'image': 'pid_57/pid_57.nii.gz', 'label': 'pid_57/pid_57_gt.nii.gz'},
  {'image': 'pid_1002/pid_1002.nii.gz',
   'label': 'pid_1002/pid_1002_gt.nii.gz'},
  {'image': 'pid_1003/pid_1003.nii.gz',
   'label': 'pid_1003/pid_1003_gt.nii.gz'}],
 'val': [{'image

## Pretrain

In [28]:
import os

workspace_dir = '/nfs/Workspace/CardiacSeg'

exp_name = 'exp_1'
model_name = 'unetcnx_x3_2_2'
data_name = 'chgh'

root_data_dir = os.path.join(workspace_dir, 'dataset', data_name)
data_dir = os.path.join(root_data_dir, 'dataset_2')
data_dicts_json = os.path.join(root_data_dir, 'data_dicts', f'exp_2_2_pretrain.json')

root_dir = os.path.join(
    workspace_dir,
    'exps',
    'exps',
    model_name,
    data_name,
    'pretrain',
    exp_name
)
model_dir = os.path.join(root_dir, 'models')
log_dir = os.path.join(root_dir, 'logs')

os.makedirs(root_dir, exist_ok=True)

best_model_path = os.path.join(model_dir, 'model_bestValRMSE.pt')
final_model_path = os.path.join(model_dir, 'model_final.pt')

%cd {root_dir}

/nfs/Workspace/CardiacSeg/exps/exps/unetcnx_x3_2_2/chgh/pretrain/exp_1


In [ ]:
!PYTHONPATH={workspace_dir} /opt/conda/bin/python {workspace_dir}/expers/pretrain/train.py\
--use_checkpoint \
--batch_size=1 \
--num_steps=100000 \
--lrdecay \
--eval_num=100 \
--max_early_stop_count=60 \
--model_name={model_name} \
--logdir={log_dir} \
--model_dir={model_dir} \
--data_dirs={data_dir} \
--data_dicts_jsons={data_dicts_json} \
--resume={final_model_path}
# --lr=<Lr> 
# --roi_x=<Roi_x> \
# --roi_y=<Roi_y> \
# --roi_z=<Roi_z> \

In [26]:
import sys
# set package path
sys.path.append(workspace_dir)

from networks.network import network
from networks.ssl_head import SSLHead

import torch

class args:
    in_channels=1
    out_channels=2
    device='cuda'

net = network(model_name, args)

model = SSLHead(
  args.in_channels,
  net, 
  dim=768
).to(args.device)

model_pth = best_model_path
model_dict = torch.load(model_pth)
model.load_state_dict(model_dict["state_dict"])

if "global_step" in model_dict:
    global_step = model_dict["global_step"]
if "early_stop_count" in model_dict:
    early_stop_count = model_dict["early_stop_count"]
if "val_best" in model_dict:
    best_val = model_dict["val_best"]
    
print('load model:', model_pth)
print('global step:', global_step)
print('best val:', best_val)

model: unetcnx_x3_2_2
load model: /nfs/Workspace/CardiacSeg/exps/exps/unetcnx_x3_2_2/chgh/pretrain/exp_1/models/pretrain/exp_1/model_bestValRMSE.pt
global step: 7100
best val: 0.0615180271367232
